<center><font size="12">Connections Example</font> </center>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd

In [3]:
from src.shared_code import connectors as conn, dataset, utils
from src import data_assets, project_env

This notebook aims at providing unit examples on `data connections` creation and using.

Firstly, `data connections` are created manually, by intanciating a `connection client` (using a factory function) that wraps the actual storage client system; the `data connectiion` is established by setting actual file metadata (like path)

Then, `data connections` are created with its on factory function

In [4]:
project_env.set_env_vars()  # laod environment variables

In [5]:
folder = "testing_io"
ext = "csv"

file_in = "test_reading"
file_out = "test_writting"

In [6]:
path_in = utils.path_data_raw / folder / f"{file_in}.{ext}"
path_out = utils.path_data_interim / folder/ f"{file_out}.{ext}"

In [7]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'c:/Users/MRomero/Documents/GCP/mlops-training-378617-49fbdb08c3f3.json'

# Prerequisites

* Define the necesary environment variables to instanciate the data connection desired, in this case, a local one. Other data connectors many need other kind of pre-requisites, like implementing a credentials instance
    * AZ_STGACC_KEY="..."
    * GOOGLE_APPLICATION_CREDENTIALS="xxx.json"
    * GCP_PROJECT=""
    * GCP_BUCKET=""
* the file `data/raw/testing_io/test_reading.csv` is created

In [8]:
path_folder = utils.path_data_raw / folder

if not (path_folder).is_dir():
    print(f"Creating folder: {path_folder}")
    path_folder.mkdir(parents=True)
    

In [9]:
data = pd.DataFrame([[1,2,3], [4,5,6]], columns=["x1", "x2", "x3"])

In [10]:
print(f"{path_in.as_posix()}")

C:/Users/MRomero/Documents/repos/labs/repo_template/data/raw/testing_io/test_reading.csv


In [11]:
data.to_csv( path_in, index=False)

# Local Connection

In [12]:
conn.connectionsEnum.LOCAL

<connectionsEnum.LOCAL: 'local'>

In [13]:
client_local = conn.connection_client_factory(
    connection_kind=conn.connectionsEnum.LOCAL,
    connection_name="conn_local")

conn_local = conn.DataConnectorLocal(
    connector_client=client_local,
    layer="raw",
    path=f"{folder}/{file_in}.{ext}"
)

dataset_local = dataset.DatasetPandas(conn_local)
df_train_local = dataset_local.read(func=pd.read_csv)

df_train_local.head()

,x1,x2,x3
0,1,2,3
1,4,5,6


In [14]:
dataset_local.metadata()

{'features': [],
 'target': [],
 'id_cols': [],
 'split_col': [],
 'features_num': [],
 'features_cat': []}

In [15]:
import sys

sys.getsizeof(dataset_local)

48

In [16]:
df_train_local.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   x1      2 non-null      int64
 1   x2      2 non-null      int64
 2   x3      2 non-null      int64
dtypes: int64(3)
memory usage: 176.0 bytes


# AzBlob connection

In [17]:
conn.connectionsEnum.AZBLOB

<connectionsEnum.AZBLOB: 'azblob'>

In [18]:
client_azblob = conn.connection_client_factory(
    connection_kind=conn.connectionsEnum.AZBLOB,
    connection_name="conn_azblob")

In [19]:
utils.path_data_raw

WindowsPath('C:/Users/MRomero/Documents/repos/labs/repo_template/data/raw')

In [20]:
conn_azblob = conn.DataConnectorAzBlob(
    connector_client=client_azblob,
    layer="raw",
    path="dataset/2022/01/dataset.csv")

conn_azblob

In [21]:
dataset_azblob = dataset.DatasetPandas(conn_azblob)
df_azblob = dataset_azblob.read(func=pd.read_csv)

df_azblob.head()

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,63,46,services,married,high.school,no,yes,no,cellular,apr,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
1,80,54,technician,married,professional.course,unknown,yes,no,cellular,apr,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
2,83,57,retired,married,university.degree,no,yes,no,cellular,apr,...,1,2,1,success,-1.8,93.075,-47.1,1.365,5099.1,0
3,100,37,management,married,university.degree,unknown,no,yes,cellular,apr,...,1,999,1,failure,-1.8,93.075,-47.1,1.405,5099.1,0
4,104,33,services,single,basic.9y,no,yes,no,cellular,apr,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.410,5099.1,0


# GCS Connection

In [22]:
conn.connectionsEnum.GCS

<connectionsEnum.GCS: 'gcs'>

In [23]:
client_gcs = conn.connection_client_factory(
    connection_kind=conn.connectionsEnum.GCS,
    connection_name="conn_gcs")

In [24]:
conn_gcs = conn.DataConnectorGCS(
    connector_client=client_gcs,
    layer="raw",
    path=f"dataset/2022/01/dataset.csv"
)

conn_gcs

In [25]:
ds_gcs = dataset.DatasetPandas(conn_gcs)
df_gcs = ds_gcs.read(func=pd.read_csv)

df_gcs.head()

data/raw/dataset/2022/01/dataset.csv


,id_row,x_age,x_job,x_marital,x_education,x_default,x_housing,x_loan,x_contact,x_month,...,x_pdays,x_previous,x_poutcome,x_emp.var.rate,x_cons.price.idx,x_cons.conf.idx,x_euribor3m,x_nr.employed,id_row.1,y_subs
0,27974,43,self-employed,married,high.school,unknown,yes,no,cellular,apr,...,999,0,nonexistent,-1.8,93.075,-47.1,1.498,5099.1,27974,0
1,27975,31,technician,married,professional.course,no,yes,no,cellular,apr,...,999,0,nonexistent,-1.8,93.075,-47.1,1.498,5099.1,27975,0
2,27976,43,self-employed,married,high.school,unknown,no,no,cellular,apr,...,999,0,nonexistent,-1.8,93.075,-47.1,1.498,5099.1,27976,0
3,27978,29,admin.,married,university.degree,no,yes,no,cellular,apr,...,999,0,nonexistent,-1.8,93.075,-47.1,1.498,5099.1,27978,0
4,27981,33,blue-collar,married,professional.course,no,yes,yes,cellular,apr,...,999,0,nonexistent,-1.8,93.075,-47.1,1.498,5099.1,27981,0


# Data Connections Factory

## Define Sources

In [26]:
source_local = data_assets.sources[0]
source_local

DataAsset(name='testing_source', kind='local', layer='raw', path='testing_io/test_reading', extension='csv', description='features and labels from UCI datasets')

## Define Sinks

In [27]:
sink_local = data_assets.sinks[0]
sink_local

DataAsset(name='testing_sink', kind='local', layer='raw', path='testing_io/test_writting', extension='csv', description='features and labels from UCI datasets')

## Define Data Connections

In [28]:
conn_fac_local_source = conn.data_connections_factory(source_local)
conn_fac_local_sink = conn.data_connections_factory(sink_local)

## Read Data

In [29]:
ds_src_raw_local = dataset.DatasetPandas(conn_fac_local_source)
df_in_local = ds_src_raw_local.read(func=pd.read_csv)

df_in_local.head()

,x1,x2,x3
0,1,2,3
1,4,5,6


## Write Data

In [30]:
ds_sink_raw_local = dataset.DatasetPandas(conn_fac_local_sink)

In [31]:
ds_sink_raw_local.write(
    df_train_local, kwargs={"index_label": False})